In [14]:
!nvidia-smi

Tue Sep 23 00:04:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              4W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# !pip install git+https://github.com/RyanWangZf/MedCLIP.git
%pip install pyarrow==14.0.2 datasets==2.16.1

In [ ]:
%pip install kagglehub datasets

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os
import pandas as pd

d:\Conda\envs\entrep\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Conda\envs\entrep\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Conda\envs\entrep\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Conda\envs\entrep\lib\site-packages\ipykernel_launcher.py", line 18, in <

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Conda\envs\entrep\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Conda\envs\entrep\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Conda\envs\entrep\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Conda\envs\entrep\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Conda\envs\entrep\lib\site-packages\ipykernel\kern

AttributeError: _ARRAY_API not found

In [4]:
%cd MedCLIP

d:\codePJ\uit-final-thesis\Vision-language-Models-in-Medical-Image-Analysis\MedCLIP


# Chexpert

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]

# # Download the dataset and explore available files
# print("Downloading CheXpert dataset...")
# dataset_path = kagglehub.dataset_download('ashery/chexpert')
# print(f"Dataset downloaded to: {dataset_path}")

chexpert_val_csv = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "ashery/chexpert",
    "valid.csv",
)

In [ ]:
def create_chexpert_5x200(chexpert_val_csv):

    condition_columns = ['Atelectasis', 'Cardiomegaly', 'Edema', 'Pleural Effusion', 'Consolidation']

    other_cols = [col for col in condition_columns if col != 'Atelectasis']
    unique_Atelectasis = chexpert_val_csv[(chexpert_val_csv['Atelectasis'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Cardiomegaly']
    unique_Cardiomegaly = chexpert_val_csv[(chexpert_val_csv['Cardiomegaly'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Edema']
    unique_Edema = chexpert_val_csv[(chexpert_val_csv['Edema'] != 0) & 
                            (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Pleural Effusion']
    unique_PleuralEffusion = chexpert_val_csv[(chexpert_val_csv['Pleural Effusion'] != 0) & 
                                        (chexpert_val_csv[other_cols] == 0).all(axis=1)]

    other_cols = [col for col in condition_columns if col != 'Consolidation']
    unique_Consolidation = chexpert_val_csv[(chexpert_val_csv['Consolidation'] != 0) & 
                                    (chexpert_val_csv[other_cols] == 0).all(axis=1)]
    return unique_Atelectasis, unique_Cardiomegaly, unique_Edema, unique_PleuralEffusion, unique_Consolidation

# MIMIC CXR

In [ ]:
from datasets import load_dataset

ds = load_dataset("itsanmolgupta/mimic-cxr-dataset")

In [ ]:
ds

# COVID 19

In [3]:
%cd MedCLIP

d:\codePJ\uit-final-thesis\Vision-language-Models-in-Medical-Image-Analysis\MedCLIP


## Prepare dataset

In [4]:
covid_data_path = kagglehub.dataset_download('tawsifurrahman/covid19-radiography-database')

  1%|▏         | 10.0M/778M [00:02<03:07, 4.29MB/s]


KeyboardInterrupt: 

In [ ]:
def create_covid_csv(covid_lst, non_covid_lst):
    df = pd.DataFrame(columns=['imgpath', 'COVID', 'Normal'])
    for covid in covid_lst:
        new_row = pd.DataFrame({'imgpath': [covid], 'COVID': [1], 'Normal': [0]})
        df = pd.concat([df, new_row], ignore_index=True)
    for non_covid in non_covid_lst:
        new_row = pd.DataFrame({'imgpath': [non_covid], 'COVID': [0], 'Normal': [1]})
        df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv('local_data/covid-test-meta.csv', index=True)
    return df

In [ ]:
covid_prompt = "the presence of patchy or confluent, bandlike ground-glass opacity or consolidation in a peripheral and mid to lower lung zone distribution"

In [ ]:
covid_data_path = os.path.join(covid_data_path, 'COVID-19_Radiography_Dataset')

In [ ]:
covid_data_path

In [ ]:
os.listdir(
    covid_data_path
)

In [ ]:
viral_pneumonia_path = os.path.join(
        covid_data_path,
        'Viral Pneumonia',
        'images'
)
viral_pneumonia_path_list = [os.path.join(viral_pneumonia_path, f) 
                            for f in os.listdir(viral_pneumonia_path)]

lung_opacity_path = os.path.join(
        covid_data_path,
        'Lung_Opacity',
        'images'
)
lung_opacity_path_list = [os.path.join(lung_opacity_path, f)
                         for f in os.listdir(lung_opacity_path)]

normal_path = os.path.join( 
        covid_data_path,
        'Normal',
        'images'
)
normal_path_list = [os.path.join(normal_path, f)
                   for f in os.listdir(normal_path)]

covid_path = os.path.join(
        covid_data_path,
        'COVID',
        'images'
)
covid_path_list = [os.path.join(covid_path, f)
                  for f in os.listdir(covid_path)]

In [ ]:
covid_csv = create_covid_csv(covid_path_list, 
                            viral_pneumonia_path_list + lung_opacity_path_list + normal_path_list)

## Zero shot classification

In [ ]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, PromptClassifier
from medclip.dataset import ZeroShotImageDataset, ZeroShotImageCollator
from medclip.evaluator import Evaluator
from medclip.prompts import generate_covid_class_prompts
from medclip.modeling_medclip import PromptClassifier
from torch.utils.data import DataLoader

In [ ]:
# Generate COVID class prompts
covid_prompts = generate_covid_class_prompts(n=5)
print("COVID class prompts:", covid_prompts)


In [ ]:
class_names = ['COVID', 'Normal']

covid_clf_dataset = ZeroShotImageDataset(
    datalist=['covid-test'],
    class_names=class_names,
    imgtransform=None
)

# Create the collator with COVID prompts
covid_collator = ZeroShotImageCollator(
    mode='binary',
    cls_prompts=covid_prompts,
    n_prompt=5
)

covid_dataloader= DataLoader(
    covid_clf_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,  # Set to 0 to avoid multiprocessing issues
    collate_fn=covid_collator
)

resnet_clf = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
resnet_clf.from_pretrained()
resnet_clf.cuda()

covid_classifier = PromptClassifier(
    medclip_model=resnet_clf,
    ensemble=True  
)

evaluator = Evaluator(
    medclip_clf=covid_classifier,
    eval_dataloader=covid_dataloader_fixed,
    mode='binary'
)

In [ ]:
print("Testing the dataloader...")
test_batch = next(iter(covid_dataloader))
print("Batch keys:", test_batch.keys())
print("Pixel values shape:", test_batch['pixel_values'].shape)
print("Labels shape:", test_batch['labels'].shape)
print("Prompt inputs keys:", test_batch['prompt_inputs'].keys())

In [ ]:
print("\nRunning evaluation...")
results = evaluator.evaluate()

In [ ]:
print(f"Results: {results}")
print(f"accuracy: {results['acc']}")
print(f"precision: {results['precision']}")
print(f"recall: {results['recall']}")
print(f"f1_score: {results['f1-score']}")


# RNSA 

## Zero shot classification

In [1]:
%cd MedCLIP

d:\codePJ\uit-final-thesis\Vision-language-Models-in-Medical-Image-Analysis\MedCLIP


In [7]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, PromptClassifier, MedCLIPVisionModel
from medclip.dataset import ZeroShotImageDataset, ZeroShotImageCollator
from medclip.evaluator import Evaluator
from medclip.prompts import generate_rsna_class_prompts
from medclip.modeling_medclip import PromptClassifier
from torch.utils.data import DataLoader

In [3]:
# Generate RSNA class prompts
rsna_prompts = generate_rsna_class_prompts(n=5)
print("RSNA class prompts:", rsna_prompts)

sample 5 num of prompts for Pneumonia from total 312
RSNA class prompts: {'Pneumonia': ['small bacterial at the left lung zone', 'early viral at the bilateral lung bases', 'early viral at the mid lung zone', 'focal mycoplasma at the lung bases', 'multifocal bacterial at the right lung base']}


In [8]:
class_names = ['Pneumonia', 'Normal']

# Use the DICOM-compatible dataset class for RSNA data
rsna_clf_dataset = ZeroShotImageDataset(
    datalist=['rsna-balanced-test'],
    class_names=class_names,
    imgtransform=None
)

# Create the collator with RSNA prompts
rsna_collator = ZeroShotImageCollator(
    mode='binary',
    cls_prompts=rsna_prompts,
    n_prompt=5
)

rsna_dataloader= DataLoader(
    rsna_clf_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,  # Set to 0 to avoid multiprocessing issues
    collate_fn=rsna_collator
)

resnet_clf = MedCLIPModel(vision_cls=MedCLIPVisionModel)
resnet_clf.from_pretrained()
resnet_clf.cuda()

rsna_classifier = PromptClassifier(
    medclip_model=resnet_clf,
    ensemble=True  
)

evaluator = Evaluator(
    medclip_clf=rsna_classifier,
    eval_dataloader=rsna_dataloader,
    mode='binary'
)

load data from ./local_data/rsna-balanced-test-meta.csv


d:\Conda\envs\entrep\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Conda\envs\entrep\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Conda\envs\entrep\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
d:\Conda\envs\entrep\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load


 Download pretrained model from: https://storage.googleapis.com/pytrial/medclip-pretrained.zip


d:\codePJ\uit-final-thesis\Vision-language-Models-in-Medical-Image-Analysis\MedCLIP\medclip\modeling_medclip.py:184: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict

load model weight from: ./pretrained/medclip-resnet


In [9]:
print("Testing the dataloader...")
test_batch = next(iter(rsna_dataloader))
print("Batch keys:", test_batch.keys())
print("Pixel values shape:", test_batch['pixel_values'].shape)
print("Labels shape:", test_batch['labels'].shape)
print("Prompt inputs keys:", test_batch['prompt_inputs'].keys())

Testing the dataloader...
Batch keys: dict_keys(['pixel_values', 'prompt_inputs', 'labels'])
Pixel values shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Prompt inputs keys: dict_keys(['Pneumonia'])


In [6]:
# Run RSNA evaluation
print("\nRunning RSNA evaluation...")
try:
    results = evaluator.evaluate()
    print(f"Results: {results}")
    print(f"accuracy: {results['acc']}")
    print(f"precision: {results['precision']}")
    print(f"recall: {results['recall']}")
    print(f"f1_score: {results['f1-score']}")
except Exception as e:
    print(f"Evaluation failed with error: {e}")
    import traceback
    traceback.print_exc()



Running RSNA evaluation...


Evaluation: 100%|██████████| 945/945 [19:41<00:00,  1.25s/it]

Results: {'pred': array([[0.2345862 ],
       [0.37272498],
       [0.30750567],
       ...,
       [0.28917682],
       [0.2749958 ],
       [0.3268345 ]], dtype=float32), 'labels': array([1, 1, 1, ..., 1, 1, 1], dtype=int64), 'auc': 0.3963176009270105, 'acc': 0.6838918847388097, 'precision': 0.34194594236940484, 'recall': 0.5, 'f1-score': 0.40613764514037604}
accuracy: 0.6838918847388097
precision: 0.34194594236940484
recall: 0.5
f1_score: 0.40613764514037604



d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [10]:
# Run RSNA evaluation
print("\nRunning RSNA evaluation...")
try:
    results = evaluator.evaluate()
    print(f"Results: {results}")
    print(f"accuracy: {results['acc']}")
    print(f"precision: {results['precision']}")
    print(f"recall: {results['recall']}")
    print(f"f1_score: {results['f1-score']}")
except Exception as e:
    print(f"Evaluation failed with error: {e}")
    import traceback
    traceback.print_exc()



Running RSNA evaluation...


Evaluation: 100%|██████████| 945/945 [16:14<00:00,  1.03s/it]

Results: {'pred': array([[-1.2724408 ],
       [-1.3392502 ],
       [-0.9474664 ],
       ...,
       [-0.5673434 ],
       [-0.48692712],
       [-0.9549953 ]], dtype=float32), 'labels': array([1, 1, 1, ..., 1, 1, 1], dtype=int64), 'auc': 0.37604737492162854, 'acc': 0.3161081152611903, 'precision': 0.15805405763059516, 'recall': 0.5, 'f1-score': 0.24018400281534363}
accuracy: 0.3161081152611903
precision: 0.15805405763059516
recall: 0.5
f1_score: 0.24018400281534363



d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Conda\envs\entrep\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
